In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor, create_openai_functions_agent, Tool


import os
from dotenv import load_dotenv

load_dotenv(override=True)
#os.environ["OPENAI_API_KEY"] = "your key"


@tool
def build(commit_id: int):
    """build artifact with commit_id ."""
    print(f"Building artifact with commit_id {commit_id}")
    build_id = 100
    return build_id

@tool
def deploy(build_id: int):
    """deploy artifact with build_id. """
    print(f"Deploying artifact with build_id {build_id}")
    deploy_id = 200
    return deploy_id

@tool
def test(deploy_id: int):
    "test artifact with deploy_id."
    print(f"Testing artifact with deploy_id {deploy_id}")
    test_id = 300
    return test_id

@tool
def report(test_id: int):
    "generate report with test_id."
    print(f"Generating report with test_id {test_id}")
    report_id = 400
    return report_id

#build.invoke({"commit_id": 1})

from operator import itemgetter
from typing import Union

from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
# Get the prompt to use 
# https://github.com/hwchase17/langchain-hub?tab=readme-ov-file
#prompt = load_prompt('lc://prompts/conversation/prompt.json')

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(
            variable_name="chat_history",optional=True
        ),
        ("human", "{input}"),
        MessagesPlaceholder(
            variable_name="agent_scratchpad",
        ),
    ]
)

llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4-0125-preview")
llm_chain = LLMChain(llm=llm, prompt=prompt)
tools = [build, deploy, test, report, 
         Tool(
             name="reason",
             func=llm_chain.invoke,
             description = "assistant to help you with the reason",
         ),]



In [8]:
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react")


# Construct the OpenAI Tools agent
#agent = create_openai_tools_agent(llm, tools, prompt)
agent = create_react_agent(llm, tools, prompt)


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "build 256 and generate report"
    }
)



> Entering new AgentExecutor chain...
Thought: I need to follow the steps from building with the commit_id, deploying the build artifact, testing the deployment, and finally generating a report with the test_id.
Action: build
Action Input: 256Building artifact with commit_id 256
100Action: deploy
Action Input: 100Deploying artifact with build_id 100
200Action: test
Action Input: 200Testing artifact with deploy_id 200
300Action: report
Action Input: 300Generating report with test_id 300
400I now know the final answer
Final Answer: The report generated for build 256 is report 400.

> Finished chain.


{'input': 'build 256 and generate report',
 'output': 'The report generated for build 256 is report 400.'}